# Potencjały wywołane ERP

* N100 (N1) - odzwierciedla wczesne przetwarzanie bodźców sensorycznych, stosowany w badaniach uwagi i selekcji bodźców.
* P300 (P3) - odzwierciedla procesy uwagi i aktualizacji pamięci roboczej, szeroko stosowany w badaniach poznania i neurologii klinicznej.
* N400 - odzwierciedla przetwarzanie semantyczne i rozbieżności w kontekście językowym, używany w badaniach nad językiem i pamięcią.
* P600 - związany z przetwarzaniem składniowym w języku, wykorzystywany w analizie zdolności językowych i zaburzeń mowy.

### Analiza potencjałów ERP (kroki)

1. Filtracja
2. Segmentacja
3. Uśrednianie sygnału

Więcej informacji: https://www.researchgate.net/post/How_to_evaluate_N170_N200_P300_N400_and_P600_from_EEG_signals

### Zacznijmy od importu potrzebnych paczek. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import aseegg as ag 
import numpy as np
import random

## 1) Wczytaj dane i zobacz surowy sygnał
### Plik, który będziemy używać to 'dane_potencjaly.csv'

In [ ]:
df = pd.read_csv('dane_potencjaly.csv', names=['FCz', 'Cz', 'Fz', 'Pz', 'C3', 'C4', 'F3', 'F4','event'])

In [ ]:
df

<img src="../images/elektrody_eeg.png" width="500" align="center">

(Kaongoen & Jo, 2017)

###   Zadanie 1 <img src="../images/pencil.png" width="20" align="left">

Narysuj sygnał z elektrody **Cz**. Oceń wzrokowo, czy widzisz **szum 50 Hz** lub **duże artefakty**. Częstotliwość próbkowania to 250. 

# 2) Filtracja danych

In [ ]:
przef = ag.gornoprzepustowy(df['Cz'],250,0.1)
przef2 = ag.pasmowozaporowy(przef,250,48,52)
przef3 = ag.pasmowoprzepustowy(przef2,250,1,30)

In [ ]:
przef3

In [ ]:
t_axis = np.linspace(0, len(przef3)/250, len(przef3))
plt.plot(t_axis,przef3)
plt.xlabel('Czas [s]')
plt.ylabel('[uV]')

# 3) Skąd pochodzą te dane (opis zadania)

<img src="../images/TRex-RAS.png" width="600" align="center">

### Opis eventów
1 - successful click - udane wciśnięcie spacji, skutkujące skokiem w grze (gra trwała dalej) <br>
2 - unsuccessful click - nieudane wciśnięcie spacji, program nie wykonał polecenia skoku (gra trwała dalej)<br>
3 - successful click before endgame - udane wciśnięcie spacji w programie, które prowadziło do przegranej gry (błąd użytkownika)<br>
4 - unsuccessful click before endgame - nieudane wciśnięcie spacji, które prowadziło do przegranej gry (błąd interfejsu)<br>
5 - succesful click endgame - moment kolizji w grze, który był poprzedzony udanym wciśnięciem spacji (feedback o błędzie)<br>
6 - unsuccesful click endgame - moment kolizji w grze, który był poprzedzony nieudanym wciśnięciem spacji

## 4) Zdarzenia (triggery): znajdź początki
W kolumnie 'event' pojawiają się flagi, kiedy występowało interesujące nas zdarzenie. Nas interesuje nr 4(unsuccessful click before endgame).

In [ ]:
# wyszukujemy indeksy w których pojawia się flaga '4', a wcześniejszy indeks ma flagę '0' 
poczatki_zdarzen = []
for i in range(len(df)):
    if df['event'][i]==4 and df['event'][i-1]==0:
        poczatki_zdarzen.append(i)

In [ ]:
#sprawdzamy liczbę takich zdarzeń
len(poczatki_zdarzen)

In [ ]:
poczatki_zdarzen

###   Zadanie 2 <img src="../images/pencil.png" width="20" align="left">

Policz odstępy w sekundach między zdarzeniami. Czy są jakieś, które wydają się niepokojące (błędne)?

# Epokowanie
### Stwórzmy 1-sekundowe wykresy dla naszych zdarzeń

In [ ]:
indeks = poczatki_zdarzen[0]
fragment_sygnalu = przef3[indeks:indeks+250]     # 250 próbek czyli 1 sekunda

t = np.linspace(0,1,250)
plt.plot(t,fragment_sygnalu)

### Czasem dla pojedynczych przypadków załamki mogą być niewidoczne. Dlatego uśrednia się sygnał z kilku zdarzeń, żeby wzmocnić szukany efekt. 

In [ ]:
sredni_sygnal = np.zeros(250)

for i in range(10):
    sredni_sygnal += przef3[poczatki_zdarzen[i]:poczatki_zdarzen[i]+250]
sredni_sygnal/=10

In [ ]:
t = np.linspace(0,1,250)
plt.plot(t,sredni_sygnal)

### Dane, które zostały dodatkowo "oczyszczone":
<img src="../images/p300n400.png" width="1000" align="center">

###   Zadanie 3 <img src="../images/pencil.png" width="20" align="left">
Policz średnie dla P300.

* Przedziały czasu, w którym spodziewany jest komponent: P300: 0.25–0.45 s

* Przelicz te zakresy na próbki (przy częstotliwości próbkowania 250 Hz) i oblicz średnie wartości sygnału sredni_sygnal w tych oknach, korzystając z funkcji np.mean

* Zapisz uzyskane średnie amplitudy i sprawdź, czy:

    - P300 ma wartość dodatnią,

* Krótko zinterpretuj wyniki – czy w sygnale widać obecność fal P300 ?

In [ ]:
m_p300 = np.mean()


### Dla porównania losowe fragmenty z sygnału

In [ ]:
losowe_indeksy = [68688, 50571, 58788, 95741, 19578, 15622, 95932, 9387, 30013, 93260]

# losowanie 10 indeksów
# losowe_indeksy = [random.randint(0,len(df)-250) for i in range(10)]

suma = np.zeros(250)
for indeks in losowe_indeksy:
    suma += przef3[indeks:indeks+250]
suma/=10

t = np.linspace(0,1,250)
plt.plot(t,suma)

###   Zadanie 4 <img src="../images/pencil.png" width="20" align="left">

Czy wykrywanie P300 przy pomocy średniej działa dla losowych fragmentów sygnału?


###   Zadanie 5 <img src="../images/pencil.png" width="20" align="left">

Rozpoznanie komponentu N400  

W pliku **`sygnal_zadanie.csv`** znajdują się dane EEG z kilku prób.  
Każdy z trzech zestawów indeksów (1, 2 i 3) odpowiada innym fragmentom sygnału związanym z reakcją na bodziec.  

Twoje zadanie:  
1. Wczytaj dane z pliku i uśrednij okna sygnałów osobno dla każdego zestawu indeksów.  
2. Dla każdego zestawu policz **średnią amplitudę w oknie N400 (0.30–0.50 s)**, korzystając z funkcji `mean()`.  
3. Porównaj uzyskane wartości – im bardziej **ujemna średnia**, tym silniejszy efekt N400.  
4. Wskaż, **który zestaw najbardziej przypomina N400** (czyli ma największą ujemną amplitudę).  

Dla lepszego porównania możesz narysować trzy średnie sygnały na jednym wykresie.  

In [ ]:
df = pd.read_csv('sygnal_zadanie.csv')

In [ ]:
indeksy_1 = [6423, 28964, 39804, 61360, 77226, 92327, 101265, 114662, 116682, 123936]
indeksy_2 = [938, 13371, 22252, 31128, 54798, 92047, 98816, 102843, 103273, 123659]
indeksy_3 = [7262, 19641, 27905, 39668, 41581, 72379, 76208, 86419, 112203, 120647]